In [1]:
import pandas as pd
import os
import numpy as np

import anthropic
from openai import OpenAI

import base64
import httpx
import requests
import json
import time

from dotenv import load_dotenv, find_dotenv
from IPython.display import Image,display

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

BLUE = '\033[94m'
END = '\033[0m' 

load_dotenv(find_dotenv())

True

In [2]:
openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
claude_client = anthropic.Anthropic(api_key=os.environ["CLAUDE_KEY"])

In [3]:
def create_prompt_1(question):
    prompt = '''
            CONTEXT:
            You are a chatbot that answers questions. 
            Answer do not include disclaimer.
            Answer should have a dry tone.
            
            QUESTION:
            Answer to this question: 
            {question}
            
            OUTPUT:
            Output is JSON object.

            [
                {{"result": "answer"}}
            ]
            
            '''
    prompt = prompt.format(question=question)
    return prompt


def create_prompt_2(item):
    prompt = '''
            CONTEXT:
            You are a data analyst API capable of item analysis.
            Please list the semantic queries that would be used to find the given item {item}.
            Answer do not include disclaimer.
            Answer do not include the item itself.
            Answer should have a dry tone.
            
            OUTPUT:
            Output is JSON object.

            [
                {{"result": ["semantic queries 1",
                             "semantic queries 2",
                             ...,
                             "semantic queries n",
                            ]}}
            ]
            
            '''
    prompt = prompt.format(item=item)
    return prompt


def create_prompt_3(text):
    prompt = '''
            CONTEXT:
            For the given short story give me brief summary.

            {text}
            Just do it, so the T-shirts say. Just pick up the gun, pull the trigger—but maybe aim first, aim at the upper sternum and then pull the trigger, congratulating yourself that at last, in your long, passive life, you have shot somebody dead. 
            So she did, and thus she became a murderer. She slipped through the night after that and disappeared into the willows to wash off any blood that spattered onto her clothing. 
            The willows were thickly interlaced, with an overpowering smell, acrid and watery. The stream flowed through and among them. She washed her hands and face, even her elbows, and pushed her way through the restraining arms of willows wherever she could find a gap. The ditches were deep. Sometimes she put a foot down in one and fell, grabbing at willows to drag herself back up again so she could keep running.
            How long could she run, and where? Pretty soon the willows ended, by the bank of the creek that was swift, dark, bitter cold and deep. Can't run there. Run down along the shore, scraping under a barbwire fence, not running but walking or stumbling, because there seemed no rhyme or reason for the contour of the ground in the dark. There was faint moonlight, from a half moon; she was afraid she would be seen in it, even though she didn't know if there was anyone to see her. 
            Had there been another besides the one she shot? He couldn't have been alone; she was sure she had heard them talking—short grunts of laughter, murmurs of discovery. Where was the other? Was it possible he was out looking for her? He'd be scrambling, that's for sure, not knowing the territory, confused and annoyed by the scratching of resistant willow twigs that sprang back and slapped the flesh.
            Her shoes were wet and heavy. Every step sucked mud. The sound of it traveled through the quiet night until she couldn't imagine she wouldn't be caught. If he caught her, she was dead. Probably worse than dead. It didn't pay to think a lot about how much worse than dead a thing could be. She needed all her thought to be on escape. In whispers she told herself how she was coming along, kept a running commentary on what a good job she was doing, and gave herself warnings about dangerous footing or sudden patches of open space.
            She thought about how she hadn't brought the gun. Maybe not bringing it meant he would have it now. But she couldn't have gotten as far as she had if she'd been saddled with trying to work the rifle barrel through these branches. And there was still one bullet in it, so she might have shot it off, shot her own foot, shot an advertisement of her location into the night sky. But now maybe he had the rifle, and it did have one bullet in it. He would not find any stash of ammunition, because there wasn't one. The two bullets had been all. And if he came after her, he would have more trouble than she would have had traveling through the willows, since he didn't know them and she did.
            Or maybe he was a fisherman, in which case he would be perfectly familiar with willows and know how to penetrate them without becoming immovably trapped in the twist and winding of their slippery, springy fingers. Maybe he was a fisherman and a woodsman, and was right now somewhere behind her, close behind her even, holding the rifle high, worming it through the spaces like someone from a movie. When she paused, she thought she could even hear that—an occasional snap where a foot bent a branch and broke it off, an occasional wet splat as from a foot pulling out of a mud ditch.
            Then she was in the willows again herself, and the breath was so painful in her pressed lungs there was no room for another. She had to stop. She could and should stop. It would give her a chance to listen and know for sure. She was safe here in this thicket, as safe as a deer, as safe as a fawn curled among the branches while its mother went away. She opened her mouth in a big round O and let the breaths come in, large and noiseless, go out the same way. Not quite noiseless; there was a shaky panting sound she couldn't repress. But it might not be heard over the white noise of the creek straining through the willows.
            And now that she wasn't running, she had to think. She had to remember. It made her eyes dilate and water, to remember the horror she'd felt when first she heard them walking across the gravel. Those grunts of laughter, murmurs of discovery, she could hear them clearly now. She could remember the creaks on the stairs from her own feet carrying her up to get the .30-30. She remembered the clucking sound as the lid popped out of the nearly empty cardboard ammo box, the dreadful click as she opened the magazine and slid the two bullets in. Closing the lever—that sound had been covered by a sound from downstairs, a sharp “hello!” Then there was the sound of them clumping from room to room, also a moving flicker from their flashlight. She thought she remembered seeing the one, all of a sudden, appearing at the top of the stairs, oh yes, she’d been pressed against the wall listening to their progress, and they’d been quiet, too, maybe scared themselves. Maybe awed by the size and emptiness of the house that wasn't theirs.
            But she never saw the other. She saw the one—first the circle of his flashlight and then the form behind it. She saw the light beam search her bedroom from side to side. She felt it cross her body and rapidly return to it, then aim directly at her face. “We're not alone,” called his voice, sing-song, and she hated the voice. It contained so much of her own fear, like a hideous magnifying mirror, and she knew without waiting another second that he would do her great harm. So she aimed at the part that would be the upper sternum because once she had seen Rick shoot a standing bear that way. When she pulled the trigger, the man made the same sound the bear had made, SHIIUUH, and the flashlight thumped across the floor. There it lay, shining on the place where the bureau leg stood on the carpet. 
            In the ensuing silence, which was staggering, she had set the gun down, picked up the flashlight, and shone it on the body. It was dead. There was no other word for it, for the way the eyes were up and sideways in a gaze no living eyes could hold. There was the blood, much of it seeping out from under the man—the boy, almost, she remembered now, a big overgrown boy, his life blood seeping onto her carpet, which was, or had been, beautiful and precious to her so that she felt an even greater anger. Anger that she had been forced to kill this kid who'd lived so short a life, anger that her carpet was horrifyingly blood-soaked, anger that she had acted from fear without weighing the choices.
            All her hair had been standing on end. That's how it felt, though it couldn't actually have been, because her hair was too long to stand on end. She remembered the surge of electricity that had come rushing through her when the deed was done, how she crouched with her mouth wide open in wonder and thought, I have killed somebody. He is lying here dead and I shot him. It was a feeling of victory beyond anything she had ever experienced—instantaneous certainty that she ruled the world and the heavens. Victory that lasted a minute or less and was replaced by sickening panic. Where was the other intruder? Was he crouching downstairs, not breathing, wondering if he was dreaming, wondering if he would be next? Had he run away?
            But she could not go down and find out. 
            Not only that, she couldn't go down and escape, because maybe he was there, waiting. Yes, probably he was there. She would have to let herself out through the window, with the twisted sheets tied together the way she'd never thought she herself would actually be doing, and she would have to do it so quietly the other wouldn’t hear or see. This she couldn’t do. 
            But she had tied the sheets and let them out the window, securing the end on the hat hook. Then she had grabbed a little suitcase, crept with it all the way to the other side of the house, and thrown it through another window, shattering the glass. If the other was still there, he would surely go in the direction of the sound, or at least his attention would. Then she tiptoed back, balanced on the edge of her own window, and let herself down the sheets as quietly as she could. When her feet touched the ground, she waited for many minutes, for maybe ten minutes, and never heard whether the other was there or not. She heard nothing. That was when she had made a run for it, across a small slope of meadow to the nearest clump of willows, praying all the way, spurt by spurt, until she reached the main growth where the willow branches enclosed and hid her.
            How could she have done this? How could she have done any of it? She had not known, so she thought, how to load the rifle. She had not known how to fire it. She had not known how to descend on a twist of sheets. There had been something working in her that was quite unknown to her conscious knowledge of herself yet apparently fully developed. Developed when, in her sleep? All the nights she had lain sleeping? Was she all that time developing the secret ability to be a killer and an escapee?
            As she paused again for breath, and to listen, she began to see it was more important at this point to run to, rather than away. Just running away meant she would have to keep on running, forever. She knew she couldn't do that. The obvious plan was to go to the Seymours for help. The Seymours were the nearest neighbors, three miles down the road. She was sure she had run more than three miles, but she didn't know how many of those miles were in a straight line. She thought, though, that the tiny flicker of light blinking occasionally behind the pines might be from their ranch. And when she came to a sweep of pasture land, she was sure of it. She began to run again because the grazed grass made it so easy, and the hope in her heart was expanding like a sunrise. A few minutes later, she staggered into their driveway, panting and heaving, and banged with her fist on the door. They were certainly home, because there was their pickup, a Dodge Ram.
            "Well, hi there, neighbor," said Ida Seymour, in that way she had of saying 'neighbor' that sounded as if she had learned it in a storybook. “Heavens, you look like you might have ran here!”
            Ida Seymour had opened the door wide for her, so she walked on into the mudroom feeling half grateful. Only half. She had another feeling, too, but it took a few seconds for her to pin it down: Do the Seymours drive a Dodge Ram? It was a nasty spark that might or might not go out when it hit the ground. She had a very strong impulse to turn around and keep running. Ida Seymour, however, was propelling her on into the kitchen, talking all the while, “You're the second visitor we've had tonight! You wouldn't guess, but we have this fellow who says there's been a killing up the creek, in an abandoned house. Willie told him, what abandoned house? There’s only the one house. Which of course is yours. Right off, we thought about you and hoped you were alright! But anyway, Willie is on the phone to the sheriff this minute. Come on in!”
            She came on in. She had to. The man at the kitchen table seemed young too, though older than the other, the dead one. His wheat-grass hair hung down all around from under the edges of his baseball cap, and his plaid flannel shirt was the shirt anyone in this part of the country would wear. He was hunched over, his hands wrapped almost desperately around a cup of black coffee. His face with its haggard blue eyes was at this moment raising toward hers, and in its clammy unhuman whiteness she saw a reflection of how her own face must look; it couldn't look any other way. A look comprised of shock, horror, revulsion, and naked fear.
            Willie Seymour stood behind him, the receiver halfway to his ear. He started to lower it, and she saw in his eyes a slow trickle of comprehension. Then that slow trickle seemed to pass from one set of eyes to another, getting less slow by the second, so that she began to fear, as she had feared in the willows. Feared more, because she couldn’t run. She had already run to the end.
            Of course she feared. She should have thought of it before. There was a murderer in this room—and everyone present knew who it was.
                        
            OUTPUT:
            Output is JSON object.
            Output must be in this form:

            [
                {{"summary": "story_summary"}}
            ]
            
            '''
    prompt = prompt.format(text=text)
    return prompt

In [4]:
def gpt_description(question, num_prompt = 1):
    
    if num_prompt == 1:
        prompt = create_prompt_1(question)
    elif num_prompt == 2:
        prompt =  create_prompt_2(question)
    elif num_prompt == 3:
        prompt =  create_prompt_3(question)
    
    response = openai_client.chat.completions.create(
      model="gpt-4-vision-preview",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            }
            
          ]
        }
      ],
      temperature=0.5,
      max_tokens=2000,
    )
    return response


def opus_description(question, num_prompt = 1):
    
    if num_prompt == 1:
        prompt = create_prompt_1(question)
    elif num_prompt == 2:
        prompt =  create_prompt_2(question)
    elif num_prompt == 3:
        prompt =  create_prompt_3(question)
    
    response = claude_client.messages.create(
                model='claude-3-opus-20240229', 
                messages=[
                  {
                    "role": "user",
                    "content": prompt
                  }
                ],
                max_tokens=2000,
                temperature=0.5,
              )
    return response


def sonnet_description(question, num_prompt = 1):
    
    if num_prompt == 1:
        prompt = create_prompt_1(question)
    elif num_prompt == 2:
        prompt =  create_prompt_2(question)
    elif num_prompt == 3:
        prompt =  create_prompt_3(question)
    
    response = claude_client.messages.create(
                model='claude-3-sonnet-20240229', 
                messages=[
                  {
                    "role": "user",
                    "content": prompt
                  }
                ],
                max_tokens=2000,
                temperature=0.5,
              )
    return response



def haiku_description(question, num_prompt = 1):
    
    if num_prompt == 1:
        prompt = create_prompt_1(question)
    elif num_prompt == 2:
        prompt =  create_prompt_2(question)
    elif num_prompt == 3:
        prompt =  create_prompt_3(question)
    
    response = claude_client.messages.create(
                model='claude-3-haiku-20240307', 
                messages=[
                  {
                    "role": "user",
                    "content": prompt
                  }
                ],
                max_tokens=2000,
                temperature=0.5,
              )
    return response

In [5]:
def cost_gpt(total, prompt_tokens, completion_tokens):
    return (((total * prompt_tokens)/1000000) * 10) + (((total * completion_tokens)/1000000) * 30)

def cost_opus(total, input_token, output_token):
  return(((total * input_token)/1000000) * 15) + (((total * output_token)/1000000) * 75)

def cost_sonnet(total, input_token, output_token):
  return(((total * input_token)/1000000) * 3) + (((total * output_token)/1000000) * 15)

def cost_haiku(total, input_token, output_token):
  return(((total * input_token)/1000000) * 0.25) + (((total * output_token)/1000000) * 1.25)

In [8]:
question = 'Who is Nikola Tesla'
num_prompt = 1

print('\n')
print(BLUE + 'GPT-4-1106-preview' + END, '\n')
st = time.time()
result_gpt = gpt_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_gpt.usage.prompt_tokens)
print('Number of output tokens:', result_gpt.usage.completion_tokens, '\n')
print('Cost for 100k queries: ', f'${cost_gpt(100000,result_gpt.usage.prompt_tokens,result_gpt.usage.completion_tokens)} $')
print('Cost per query: ', f'${cost_gpt(1,result_gpt.usage.prompt_tokens,result_gpt.usage.completion_tokens)} $','\n')
for l in result_gpt.choices[0].message.content.split('\n'):
        print(l)


print('\n')
print(BLUE + 'Opus' + END, '\n')
st = time.time()
result_opus = opus_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_opus.usage.input_tokens)
print('Number of output tokens:', result_opus.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_opus(100000,result_opus.usage.input_tokens,result_opus.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_opus(1,result_opus.usage.input_tokens,result_opus.usage.output_tokens)} $','\n')
for l in result_opus.content[0].text.split('\n'):
        print(l)


print('\n')
print(BLUE + 'Sonnet' + END, '\n')
st = time.time()
result_sonnet = sonnet_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_sonnet.usage.input_tokens)
print('Number of output tokens:', result_sonnet.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_sonnet(100000,result_sonnet.usage.input_tokens,result_sonnet.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_sonnet(1,result_sonnet.usage.input_tokens,result_sonnet.usage.output_tokens)} $','\n')
for l in result_sonnet.content[0].text.split('\n'):
        print(l)



print('\n')
print(BLUE + 'Haiku' + END, '\n')
st = time.time()
result_haiku = haiku_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_haiku.usage.input_tokens)
print('Number of output tokens:', result_haiku.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_haiku(100000,result_haiku.usage.input_tokens,result_haiku.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_haiku(1,result_haiku.usage.input_tokens,result_haiku.usage.output_tokens)} $','\n')
for l in result_haiku.content[0].text.split('\n'):
        print(l)



GPT-4-1106-preview 

Time:  2.95 seconds 

Number of input tokens: 76
Number of output tokens: 48 

Cost for 100k queries:  $220.0 $
Cost per query:  $0.0022 $ 

[
    {"result": "Nikola Tesla was a Serbian-American inventor, electrical engineer, mechanical engineer, and futurist best known for his contributions to the design of the modern alternating current (AC) electricity supply system."}
]


Opus 

Time:  7.89 seconds 

Number of input tokens: 96
Number of output tokens: 151 

Cost for 100k queries: 1276.5 $
Cost per query:  0.012765 $ 

[
    {
        "result": "Nikola Tesla was a Serbian-American inventor, electrical engineer, and futurist best known for his contributions to the design of the modern alternating current (AC) electricity supply system. Born in 1856, Tesla developed a number of groundbreaking technologies, including the Tesla coil, fluorescent lighting, the induction motor, and early X-ray imaging. He also explored the possibilities of wireless power transmissio

In [12]:
question = 'Smartphone'
num_prompt = 2


print('\n')
print(BLUE + 'GPT-4-1106-preview' + END, '\n')
st = time.time()
result_gpt = gpt_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_gpt.usage.prompt_tokens)
print('Number of output tokens:', result_gpt.usage.completion_tokens, '\n')
print('Cost for 100k queries: ', f'${cost_gpt(100000,result_gpt.usage.prompt_tokens,result_gpt.usage.completion_tokens)} $')
print('Cost per query: ', f'${cost_gpt(1,result_gpt.usage.prompt_tokens,result_gpt.usage.completion_tokens)} $','\n')
for l in result_gpt.choices[0].message.content.split('\n'):
        print(l)

print('\n')
print(BLUE + 'Opus' + END, '\n')
st = time.time()
result_opus = opus_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_opus.usage.input_tokens)
print('Number of output tokens:', result_opus.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_opus(100000,result_opus.usage.input_tokens,result_opus.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_opus(1,result_opus.usage.input_tokens,result_opus.usage.output_tokens)} $','\n')
for l in result_opus.content[0].text.split('\n'):
        print(l)


print('\n')
print(BLUE + 'Sonnet' + END, '\n')
st = time.time()
result_sonnet = sonnet_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_sonnet.usage.input_tokens)
print('Number of output tokens:', result_sonnet.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_sonnet(100000,result_sonnet.usage.input_tokens,result_sonnet.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_sonnet(1,result_sonnet.usage.input_tokens,result_sonnet.usage.output_tokens)} $','\n')
for l in result_sonnet.content[0].text.split('\n'):
        print(l)



print('\n')
print(BLUE + 'Haiku' + END, '\n')
st = time.time()
result_haiku = haiku_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_haiku.usage.input_tokens)
print('Number of output tokens:', result_haiku.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_haiku(100000,result_haiku.usage.input_tokens,result_haiku.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_haiku(1,result_haiku.usage.input_tokens,result_haiku.usage.output_tokens)} $','\n')
for l in result_haiku.content[0].text.split('\n'):
        print(l)



GPT-4-1106-preview 

Time:  5.0 seconds 

Number of input tokens: 108
Number of output tokens: 71 

Cost for 100k queries:  $321.0 $
Cost per query:  $0.00321 $ 

{
    "result": [
        "mobile device",
        "cell phone",
        "handheld communication device",
        "portable electronic device",
        "wireless communication device",
        "smart mobile phone",
        "internet-enabled device",
        "touchscreen device",
        "Android or iOS device",
        "personal digital assistant"
    ]
}


Opus 

Time:  5.44 seconds 

Number of input tokens: 133
Number of output tokens: 105 

Cost for 100k queries: 987.0 $
Cost per query:  0.00987 $ 

Here is the JSON object with semantic queries to find the item "Smartphone":

[
    {"result": [
        "mobile device with touchscreen and internet connectivity",
        "handheld computer that can make phone calls and run apps", 
        "cell phone with advanced computing capabilities",
        "pocket-sized device combi

In [11]:
question = 'KILLER IN THE WILLOWS'
num_prompt = 3


print('\n')
print(BLUE + 'GPT-4-1106-preview' + END, '\n')
st = time.time()
result_gpt = gpt_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_gpt.usage.prompt_tokens)
print('Number of output tokens:', result_gpt.usage.completion_tokens, '\n')
print('Cost for 100k queries: ', f'${cost_gpt(100000,result_gpt.usage.prompt_tokens,result_gpt.usage.completion_tokens)} $')
print('Cost per query: ', f'${cost_gpt(1,result_gpt.usage.prompt_tokens,result_gpt.usage.completion_tokens)} $','\n')
for l in result_gpt.choices[0].message.content.split('\n'):
        print(l)

print('\n')
print(BLUE + 'Opus' + END, '\n')
st = time.time()
result_opus = opus_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_opus.usage.input_tokens)
print('Number of output tokens:', result_opus.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_opus(100000,result_opus.usage.input_tokens,result_opus.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_opus(1,result_opus.usage.input_tokens,result_opus.usage.output_tokens)} $','\n')
for l in result_opus.content[0].text.split('\n'):
        print(l)


print('\n')
print(BLUE + 'Sonnet' + END, '\n')
st = time.time()
result_sonnet = sonnet_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_sonnet.usage.input_tokens)
print('Number of output tokens:', result_sonnet.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_sonnet(100000,result_sonnet.usage.input_tokens,result_sonnet.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_sonnet(1,result_sonnet.usage.input_tokens,result_sonnet.usage.output_tokens)} $','\n')
for l in result_sonnet.content[0].text.split('\n'):
        print(l)



print('\n')
print(BLUE + 'Haiku' + END, '\n')
st = time.time()
result_haiku = haiku_description(question, num_prompt)
print('Time: ', round(time.time()-st, 2),'seconds','\n')
print('Number of input tokens:', result_haiku.usage.input_tokens)
print('Number of output tokens:', result_haiku.usage.output_tokens, '\n')
print('Cost for 100k queries: 'f'{cost_haiku(100000,result_haiku.usage.input_tokens,result_haiku.usage.output_tokens)} $')
print('Cost per query: ', f'{cost_haiku(1,result_haiku.usage.input_tokens,result_haiku.usage.output_tokens)} $','\n')
for l in result_haiku.content[0].text.split('\n'):
        print(l)



GPT-4-1106-preview 

Time:  7.67 seconds 

Number of input tokens: 2869
Number of output tokens: 99 

Cost for 100k queries:  $3166.0 $
Cost per query:  $0.03166 $ 

[
    {"summary": "A woman becomes a murderer after shooting an intruder in her home. She flees into the willows to clean herself and escape, but is pursued by another intruder. She runs through the night, unsure of where to go, until she decides to seek help from her neighbors, the Seymours. Upon arrival, she realizes the other intruder is already there, pretending to be a victim, and she is trapped with no way to escape."}
]


Opus 

Time:  11.0 seconds 

Number of input tokens: 3035
Number of output tokens: 153 

Cost for 100k queries: 5700.0 $
Cost per query:  0.056999999999999995 $ 

Here is the summary of the short story in JSON format:

[
    {
        "summary": "A woman shoots and kills an intruder in her home. Panicked, she flees into the nearby willows, trying to escape through the difficult terrain while fear